The paper: LexRank: Graph-based Lexical Centrality as Salience in Text Summarization
https://www.cs.cmu.edu/afs/cs/project/jair/pub/volume22/erkan04a-html/erkan04a.html

In [96]:
from collections import Counter, defaultdict
import platform
import MeCab
import numpy as np
import re
from  urllib  import request
from tqdm import tqdm_notebook as tqdm
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.csgraph import connected_components

In [97]:
import regex
from urlextract import URLExtract

EMAIL_REGEX = regex.compile(
    r'[\p{L}0-9]+[\p{L}0-9_.+-]*[\p{L}0-9_+-]+@[\p{L}0-9]+[\p{L}0-9.-]*\.\p{L}+'  # noqa
)
PUNCTUATION_SIGNS = set('.,;:¡!¿?…⋯&‹›«»\"“”[]()⟨⟩}{/|\\')

url_extractor = URLExtract()



class Tokenizer_en:
    def __init__(self, stopwords):
        self.stopwords = stopwords
    
    def clean_text(self, text, allowed_chars='- '):
        text = ' '.join(text.lower().split())
        text = ''.join(ch for ch in text if ch.isalnum() or ch in allowed_chars)

        return text


    def contains_letters(self, word):
        return any(ch.isalpha() for ch in word)


    def contains_numbers(self, word):
        return any(ch.isdigit() for ch in word)


    def filter_words(self, words, stopwords, keep_numbers=False):
        if keep_numbers:
            words = [
                word for word in words
                if (self.contains_letters(word) or self.contains_numbers(word)) and
                word not in stopwords
            ]

        else:
            words = [
                word for word in words
                if self.contains_letters(word) and not self.contains_numbers(word) and
                word not in stopwords
            ]

        return words


    def separate_punctuation(self, text):
        text_punctuation = set(text) & PUNCTUATION_SIGNS

        for ch in text_punctuation:
            text = text.replace(ch, ' ' + ch + ' ')

        return text


    def tokenize(self, text, keep_numbers=False, keep_emails=False, keep_urls=False):
        tokens = []

        for word in text.split():
            emails = EMAIL_REGEX.findall(word)

            if emails:
                if keep_emails:
                    tokens.append(emails[0])

                continue

            urls = url_extractor.find_urls(word, only_unique=True)

            if urls:
                if keep_urls:
                    tokens.append(urls[0].lower())

                continue

            cleaned = self.clean_text(self.separate_punctuation(word)).split()
            cleaned = self.filter_words(cleaned, self.stopwords, keep_numbers=keep_numbers)

            tokens.extend(cleaned)

        return tokens

In [98]:
t = Tokenizer_en(stopwords=[])

In [99]:
t.tokenize("Mr Osborne said the coalition government was planning to change the tax system")

['mr',
 'osborne',
 'said',
 'the',
 'coalition',
 'government',
 'was',
 'planning',
 'to',
 'change',
 'the',
 'tax',
 'system']

In [100]:
class Tokenizer_ja:
    def __init__(self, stopwords=None, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
        print("build Tokenizer")
        if stopwords:
            self.stopwords = stopwords
        else:
            self.stopwords = self.get_stopwords()
        self.include_pos = include_pos if include_pos else  ["名詞", "動詞", "形容詞"]
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ["数"]
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"  # no matching reg
        if parser:
            self.parser = parser
        else:
            self.parser = self.get_parser()
            

    def tokenize(self, text, show_pos=False):
        text = re.sub(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", "", text)    #URL
        text = re.sub(r"\"?([-a-zA-Z0-9.`?{}]+\.jp)\"?" ,"", text)  # xxx.jp 
        text = text.lower()
        l = [line.split("\t") for line in self.parser(text).split("\n")]
        res = [
            i[2] if not show_pos else (i[2],i[3]) for i in l 
                if len(i) >=4 # has POS.
                    and i[3].split("-")[0] in self.include_pos
                    and i[3].split("-")[1] not in self.exclude_posdetail
                    and not re.search(r"(-|−)\d", i[2])
                    and not re.search(self.exclude_reg, i[2])
                    #and i[2] not in self.stopwords          
            ]
        return res
    
    def get_parser(self):
        if platform.system() == "Darwin":
            dic_dir = "/usr/local/lib/mecab/dic/mecab-ipadic-neologd/" #mac
        else:
            dic_dir = "/usr/lib/mecab/dic/mecab-ipadic-neologd"
        mecab = MeCab.Tagger("-Ochasen")# -d {}".format(dic_dir))
        return mecab.parse
    
    
    
    def get_stopwords(self):
        print("get stopwords from the web site.")
        res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
        stopwords = [line.decode("utf-8").strip() for line in res]
        print("Japanese stopword: ", ", ".join(stopwords[:3]), "...")
        res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
        stopwords += [line.decode("utf-8").strip() for line in res]
        print("English stopword: ...", ", ".join(stopwords[-3:]), )
        return stopwords

In [101]:
def _power_method(transition_matrix, increase_power=True):
    
    eigenvector = np.ones(len(transition_matrix))

    if len(eigenvector) == 1:
        return eigenvector

    transition = transition_matrix.transpose()

    while True:
        eigenvector_next = np.dot(transition, eigenvector)

        if np.allclose(eigenvector_next, eigenvector):
            return eigenvector_next

        eigenvector = eigenvector_next

        if increase_power:
            transition = np.dot(transition, transition)


def connected_nodes(matrix):
    _, labels = connected_components(matrix)

    groups = []

    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)

    return groups


def stationary_distribution(transition_matrix, increase_power=True, normalized=True):
    size = len(transition_matrix)
    distribution = np.zeros(size)

    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        distribution[group] = eigenvector

    if normalized:
        distribution /= size

    return distribution

In [113]:
class LexRank:
    def __init__(self, documents, tokenizer, include_new_words=True):
        self.tokenizer=tokenizer
        self.include_new_words = include_new_words
        self.idf_score = self._calculate_idf(documents)
        

    def get_summary(self, sentences, summary_size=1, threshold=.03, fast_power_method=True):
        if not isinstance(summary_size, int) or summary_size < 1:
            raise ValueError('\'summary_size\' should be a positive integer')
            
        lex_scores = self.rank_sentences(sentences, threshold=threshold,fast_power_method=fast_power_method) 
        sorted_ix = np.argsort(lex_scores)[::-1]
        summary = [sentences[i] for i in sorted_ix[:summary_size]]
        return summary

    def rank_sentences(self, sentences, threshold=.03, fast_power_method=True):
        if not (threshold is None or isinstance(threshold, float) and 0 <= threshold < 1):
            raise ValueError(" 'threshold' should be a floating-point number from the interval [0, 1) or None")
            
        tf_scores = [Counter(self.tokenize_sentence(sentence)) for sentence in sentences]
        similarity_matrix = self._calculate_similarity_matrix(tf_scores)
        
        if threshold is None:
            markov_matrix = self._markov_matrix(similarity_matrix)
        else:
            markov_matrix = self._markov_matrix_discrete(similarity_matrix, threshold=threshold)
        scores = stationary_distribution(markov_matrix, increase_power=fast_power_method, normalized=False)
        return scores

    def sentences_similarity(self, sentence_1, sentence_2):
        tf_1 = Counter(self.tokenize_sentence(sentence_1))
        tf_2 = Counter(self.tokenize_sentence(sentence_2))
        similarity = self.get_simiarity([tf_1, tf_2], 0, 1)
        return similarity

    def tokenize_sentence(self, sentence):
        tokens = self.tokenizer.tokenize(sentence)
        return tokens

    def _calculate_idf(self, documents):
        print("calculating idf")
        docs = [" ".join(sentences) for sentences in tqdm(documents)]
        tf = TfidfVectorizer(use_idf=True)
        self.tf_matrix = tf.fit_transform(docs)
        idf = tf.idf_ 
        if self.include_new_words:
            default_value = np.log(len(docs) + 1)
        else:
            default_value = 0
        idf_score = defaultdict(lambda: default_value)
        for k, v in tf.vocabulary_.items():
            idf_score[k] = idf[v] - 1        # idf = log(D/df) in the paper. no adding1.
        return idf_score
            
    def _calculate_similarity_matrix(self, tf_scores):
        length = len(tf_scores)
        similarity_matrix = np.zeros([length] * 2)
        for i in range(length):
            for j in range(i, length):
                similarity = self.get_simiarity(tf_scores, i, j)
                if similarity:
                    similarity_matrix[i, j] = similarity
                    similarity_matrix[j, i] = similarity
        return similarity_matrix
    
    def get_simiarity(self, tf_scores, i, j):
        return self._idf_modified_cosine(tf_scores, i, j)

    def _idf_modified_cosine(self, tf_scores, i, j):
        if i == j:
            return 1
        tf_i, tf_j = tf_scores[i], tf_scores[j]
        words_i, words_j = set(tf_i.keys()), set(tf_j.keys())
        nominator = sum([tf_i[word] * tf_j[word] * self.idf_score[word] ** 2 for word in words_i & words_j])
        if np.isclose(nominator, 0):
            return 0
        denominator_i = sum([(tf_i[word] * self.idf_score[word]) **2 for word in words_i])
        denominator_j = sum([(tf_j[word] * self.idf_score[word]) **2 for word in words_j])
        similarity = nominator / np.sqrt(denominator_i * denominator_j)
        return similarity

    def _markov_matrix(self, similarity_matrix):
        row_sum = similarity_matrix.sum(axis=1, keepdims=True)
        return similarity_matrix / row_sum

    def _markov_matrix_discrete(self, similarity_matrix, threshold):
        markov_matrix = np.zeros(similarity_matrix.shape)
        for i in range(len(similarity_matrix)):
            columns = np.where(similarity_matrix[i] > threshold)[0]
            markov_matrix[i, columns] = 1 / len(columns)
        return markov_matrix

In [114]:
documents = []
documents_dir = Path('../Datasets/livedoor/ライブドアニュース/texts')

for file_path in tqdm(documents_dir.glob('*.txt')):
    with file_path.open(mode='rt', encoding='utf-8') as fp:
        documents.append(fp.readlines())

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [115]:
t = Tokenizer_ja()

build Tokenizer
get stopwords from the web site.
Japanese stopword:  あそこ, あたり, あちら ...
English stopword: ... you've, z, zero


In [116]:
t.tokenize("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認める', 'もの', '自分', '自身', '若い', 'さ', '故', '過ち', 'もの']

In [117]:
lxr = LexRank(documents,tokenizer=t)

calculating idf


HBox(children=(IntProgress(value=0, max=7367), HTML(value='')))

In [130]:
#../Datasets/livedoor/ライブドアニュース/texts/dokujo-tsushin-4778030.txt
article = """
    http://news.livedoor.com/article/detail/4778030/
2010-05-22T14:30:00+0900
友人代表のスピーチ、独女はどうこなしている？
　もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？　さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない。

　「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」

　さてそんなとき、独女はどう対応したらいいか？

　最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる。しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした。一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに。そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである。

　「一晩で3人位の人が添削してくれましたよ。ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）。ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった。めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった。

　しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ。一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳）

　「私は基本的に人前で話すのが苦手なんですよ。だからいきなり指名されるとしどろもどろになって何もいえなくなる。そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」
　
　サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという楽しさがあるようだ。しかしそれも上手に対応できる人ならいいが、苦手な人の場合だと「フランク」ではなく「しどろもどろ」になる危険性大。ちなみにプロの司会者の場合、本当のサプライズではなく式の最中に「のちほどサプライズスピーチとしてご指名させていただきます」という一言があることも多いようだが、薫さん曰く「そんな何分前に言われても無理！」らしい。要は「サプライズを楽しめる」というタイプの人選が大切ということか。

　一方「ありきたりじゃつまらないし、ネットで例文を検索している際に『こんな方法もあるのか！』って思って取り入れました」という幸恵さん（30歳）が行ったスピーチは「手紙形式のスピーチ」というもの。

　「○○ちゃんへ　みたいな感じで新婦の友人にお手紙を書いて読み上げるやり方です。これなら多少フランクな書き方でも大丈夫だし、何より暗記しないで堂々と読み上げることができますよね。読んだものはそのまま友人にあげれば一応記念にもなります」（幸恵さん）
なるほど、確かにこれなら読みあげればいいだけなので、人前で話すのが苦手な人でも失敗しないかもしれない。

　主役はあくまで新郎新婦ながらも、いざとなると緊張し、内容もあれこれ考えて、こっそりリハーサル……そんな人知れず頑張るスピーチ担当独女たちにも幸あれ（高山惠）
"""

In [131]:
sentences = article.replace("\n","").replace(" ","").replace('\u3000', '').split("。")

In [132]:
sentences

['http://news.livedoor.com/article/detail/4778030/2010-05-22T14:30:00+0900友人代表のスピーチ、独女はどうこなしている？もうすぐジューン・ブライドと呼ばれる６月',
 '独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない',
 '「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」さてそんなとき、独女はどう対応したらいいか？最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる',
 'しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした',
 '一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに',
 'そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである',
 '「一晩で3人位の人が添削してくれましたよ',
 'ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）',
 'ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった',
 'めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった',
 'しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ',
 '一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳）「私は基本的に人前で話すのが苦手なんですよ',
 'だからいきなり指名されるとしどろもどろになって何もいえなくなる',
 'そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという

In [133]:
len(sentences)

22

In [134]:
# get summary with classical LexRank algorithm
summary = lxr.get_summary(sentences, summary_size=5)
#print(summary)


# get summary with continuous LexRank
summary_cont = lxr.get_summary(sentences, threshold=None)
#print(summary_cont)

# get LexRank scores for sentences
# 'fast_power_method' speeds up the calculation, but requires more RAM
scores_cont = lxr.rank_sentences(
    sentences,
    #threshold=None,
    #fast_power_method=False,
)
print(scores_cont)

[0.77297297 1.18918919 1.24864865 1.12972973 1.18918919 1.07027027
 1.01081081 1.12972973 1.07027027 0.47567568 1.12972973 0.89189189
 1.12972973 1.18918919 0.89189189 1.18918919 0.35675676 1.18918919
 0.41621622 1.12972973 1.12972973 1.07027027]


In [135]:
np.argsort(scores_cont)[::-1]

array([ 2, 15,  4,  1, 17, 13,  7, 10, 19,  3, 12, 20,  5,  8, 21,  6, 14,
       11,  0,  9, 18, 16])

In [136]:
importance_index = sorted(np.argsort(scores_cont)[::-1][:10])
importance_index

[1, 2, 3, 4, 7, 10, 13, 15, 17, 19]

In [137]:
[s for i,s in enumerate(sentences) if i in importance_index]

['独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない',
 '「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」さてそんなとき、独女はどう対応したらいいか？最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる',
 'しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした',
 '一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに',
 'ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）',
 'しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ',
 'そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという楽しさがあるようだ',
 'ちなみにプロの司会者の場合、本当のサプライズではなく式の最中に「のちほどサプライズスピーチとしてご指名させていただきます」という一言があることも多いようだが、薫さん曰く「そんな何分前に言われても無理！」らしい',
 '一方「ありきたりじゃつまらないし、ネットで例文を検索している際に『こんな方法もあるのか！』って思って取り入れました」という幸恵さん（30歳）が行ったスピーチは「手紙形式のスピーチ」というもの',
 'これなら多少フランクな書き方でも大丈夫だし、何より暗記しないで堂々と読み上げることができますよね']

In [73]:
with open("en_stopwords", "r") as f:
    stopwords = f.read()

In [74]:
stopwords= stopwords.split("\n")[:-1]

In [75]:
documents = []
documents_dir = Path("../Datasets/bbc/politics")

for file_path in documents_dir.glob('*.txt'):
    with file_path.open(mode='rt', encoding='utf-8') as fp:
        documents.append(fp.readlines())

In [76]:
t = Tokenizer_en(stopwords=stopwords)

In [56]:
lxr = LexRank(documents, tokenizer=t)

sentences = [
    'One of David Cameron\'s closest friends and Conservative allies, '
    'George Osborne rose rapidly after becoming MP for Tatton in 2001.',

    'Michael Howard promoted him from shadow chief secretary to the '
    'Treasury to shadow chancellor in May 2005, at the age of 34.',

    'Mr Osborne took a key role in the election campaign and has been at '
    'the forefront of the debate on how to deal with the recession and '
    'the UK\'s spending deficit.',

    'Even before Mr Cameron became leader the two were being likened to '
    'Labour\'s Blair/Brown duo. The two have emulated them by becoming '
    'prime minister and chancellor, but will want to avoid the spats.',

    'Before entering Parliament, he was a special adviser in the '
    'agriculture department when the Tories were in government and later '
    'served as political secretary to William Hague.',

    'The BBC understands that as chancellor, Mr Osborne, along with the '
    'Treasury will retain responsibility for overseeing banks and '
    'financial regulation.',

    'Mr Osborne said the coalition government was planning to change the '
    'tax system \"to make it fairer for people on low and middle '
    'incomes\", and undertake \"long-term structural reform\" of the '
    'banking sector, education and the welfare state.',
]



scores_cont = lxr.rank_sentences(sentences)
print(scores_cont.tolist())
print(scores_cont.tolist() == [1.0526315789473673, 0.5263157894736842, 1.0526315789473673, 1.0, 1.0, 1.3157894736842095, 1.0526315789473673])


# get summary with classical LexRank algorithm
summary = lxr.get_summary(sentences, summary_size=2)
print(summary)

summary ==  [
    'The BBC understands that as chancellor, Mr Osborne, along with the Treasury '
    'will retain responsibility for overseeing banks and financial regulation.',
    
    'Mr Osborne said the coalition government was planning to change the tax '
    'system "to make it fairer for people on low and middle incomes", and '
    'undertake "long-term structural reform" of the banking sector, education and '
    'the welfare state.'
]

calculating idf


HBox(children=(IntProgress(value=0, max=7367), HTML(value='')))


[0.9032258064516124, 0.9032258064516125, 1.1290322580645156, 0.9032258064516125, 0.6774193548387094, 1.3548387096774186, 1.1290322580645156]
False
['The BBC understands that as chancellor, Mr Osborne, along with the Treasury will retain responsibility for overseeing banks and financial regulation.', 'Mr Osborne said the coalition government was planning to change the tax system "to make it fairer for people on low and middle incomes", and undertake "long-term structural reform" of the banking sector, education and the welfare state.']


True